created 7-5-24
# <mark style="background-color: #B5F4D7"> <font color ='#245649'> KD = $1 \times 10^{-5}$ </font></mark>

# <mark style='background-color: #B5F4D7'><font color ='#245649'> Reformat drifter data \& collect northward particle data </font></mark>

This notebook is used to format raw drifters data from MOM6 simulations that were run with the KD parameter (background diapcynal diffusivity) set to 1E-5. 

Data is originally in the format with (i) coordinates and dimensions where every particle recorded, for every time step, is assigned a unique (i) value. We want to change the dimensions to (i,time) where (i) now refers to a unique drifter number for all time. 

In [25]:
import xarray as xr
from xgcm import Grid
import numpy as np
import warnings
from matplotlib import pyplot as plt
import matplotlib.ticker as tick
import cmocean.cm as cmo
import cmocean
import pandas as pd
%matplotlib inline
plt.rcParams['figure.figsize']=(8,5)

## <font color="F907035"> Format raw drifter data for merge with area </font>

In [26]:
ocean_geo = xr.open_dataset('/h/kuyeda/3-12_sb/sb_ocean_geometry.nc')
ocean_geo = ocean_geo.rename({'lath' : 'yh', 
                'lonh' : 'xh',
                'latq' : 'yq',
                'lonq' : 'xq'})
# ocean_geo

In [28]:
prog = xr.open_dataset('/h/kuyeda/3-7_KD1E-5/KD1E-5_oneper_linear_d2_prog__1624_326.nc',decode_times=True)
# prog
# create a grid
grid= Grid(prog, coords={'X':{'center':'xh','outer':'xq'},'Y':{'center':'yh','outer':'yq'}, 'Z':{'center':'zl','outer':'zi'}},periodic=['X'])

### <font color="F907035"> add `valid_time(i)` coordinate </font>

In [30]:
# open and format
KD_val = 'KD=1E-5'
# drifters0 = xr.open_dataset('/h/kuyeda/3-12_KD1E-5_drifters/4-25_KD1E-5_check_init_drifter_traj.nc',decode_times=False)
drifters0 = xr.open_dataset('/h/kuyeda/3-12_KD1E-5_drifters/7-23_TD_KD1E-5_drifter_traj.nc',decode_times=False)
drifters1 = drifters0.assign_coords(valid_time = drifters0.day + (drifters0.year * 368) - (1500*368)+12)
drifters1.valid_time.attrs.update({'long_name' : 'days since 01-01-1500'})

# lon > 80 ==> drifter is in reentrant channel ==> in southern ocean ==> can delete
drifters2 = drifters1.where(drifters1.lon <= 80).dropna('i')

# since we're comparing 60 years SB to 60 years KD1E-5, I'm just going to drop any time greater than 60 years
drifters = drifters2.where(drifters2.valid_time <= 22065).dropna('i')

drifters

<xarray.Dataset>
Dimensions:      (i: 5311106)
Coordinates:
    valid_time   (i) float64 5.0 0.06944 5.0 ... 2.206e+04 2.206e+04 2.206e+04
Dimensions without coordinates: i
Data variables:
    lon          (i) float64 4.24 4.5 3.358 3.5 ... 3.835 30.03 58.97 65.42
    lat          (i) float64 -8.729 -10.5 -8.26 -10.5 ... 65.13 67.0 65.81 62.52
    k            (i) float64 8.387 8.502 9.385 9.503 ... 17.46 0.0 10.91 9.477
    depth        (i) float64 192.7 197.5 234.0 239.5 ... 849.0 0.0 308.2 238.6
    year         (i) float64 1.499e+03 1.499e+03 ... 1.559e+03 1.559e+03
    day          (i) float64 361.0 356.1 361.0 356.1 ... 341.0 341.0 341.0 341.0
    drifter_num  (i) float64 137.0 137.0 106.0 106.0 ... 399.0 486.0 613.0 463.0
    id_cnt       (i) float64 9.0 9.0 10.0 10.0 9.0 9.0 ... 0.0 0.0 0.0 0.0 0.0
    id_ij        (i) float64 4.725e+03 4.725e+03 ... 4.74e+03 4.735e+03
    theta        (i) float64 9.68 10.82 7.211 7.956 ... 2.167 2.223 2.431 2.962
    uvel         (i) float64 0.001935 0.0 -0.0002699 0.0 ... 0.0 -0.0118 0.01216
    vvel         (i) float64 0.07521 0.0 0.08885 0.0 ... 0.0 0.003427 0.009703
Attributes:
    file_format_major_version:  0
    file_format_minor_version:  1

### <font color="F907035"> step function to regrid `lon` and `depth` datavariables onto `xh` and `zl` coordinates </font>

In [33]:
# loose step function to regrid drifter depth onto zl
def f(x):
    if 0<=x < 13:
        zl_depth = 6.5
    elif 13 <= x < 28:
        zl_depth = 20.5
    elif 28<= x < 45:
        zl_depth = 36.5
    elif 45<= x < 65:
        zl_depth = 55
    elif 65 <= x < 88:
        zl_depth = 76.5
    elif 88 <= x < 114:
        zl_depth = 101
    elif 114 <= x < 144:
        zl_depth = 129
    elif 144 <= x < 178:
        zl_depth = 161
    elif 178 <= x < 217:
        zl_depth = 197.5
    elif 217 <= x < 262:
        zl_depth = 239.5
    elif 262 <= x < 313:
        zl_depth = 287.5
    elif 313 <= x < 371:
        zl_depth = 342
    elif 371 <= x < 436:
        zl_depth = 403.5
    elif 436 <= x < 510:
        zl_depth = 473
    elif 510 <= x < 593:
        zl_depth = 551.5
    elif 593 <= x < 686:
        zl_depth = 639.5
    elif 686 <= x < 791:
        zl_depth = 738.5
    elif 791 <= x < 908:
        zl_depth = 849.5
    elif 908 <= x < 1038:
        zl_depth = 973
    elif 1038 <= x < 1182:
        zl_depth = 1110
    elif 1182 <= x < 1340:
        zl_depth = 1261
    elif 1340 <= x < 1513:
        zl_depth = 1426.5
    elif 1513 <= x < 1701:
        zl_depth = 1607
    elif 1701 <= x < 1905:
        zl_depth = 1803
    elif 1905 <= x < 2124:
        zl_depth = 2014.5
    elif 2124 <= x < 2357:
        zl_depth = 2240.5
    elif 2357 <= x < 2604:
        zl_depth = 2480.5
    elif 2604 <= x < 2863:
        zl_depth = 2733.5
    elif 2863 <= x < 3132:
        zl_depth = 2997.5
    elif 3132 <= x < 3409:
        zl_depth = 3270.5
    elif 3409 <= x < 3691:
        zl_depth = 3550
    elif 3691 <= x < 4000:
        zl_depth = 3845.5
    return zl_depth

In [34]:
driftersround_for_xh = drifters.assign(xh=(np.ceil(drifters.lon) - 0.5))
drifters_xh_zl1 = driftersround_for_xh.assign(zl_appx = xr.apply_ufunc(f,drifters.depth,vectorize=True))
drifters_xh_zl2 = drifters_xh_zl1.assign(depth_gridded = xr.apply_ufunc(f,drifters.depth,vectorize=True))
drifters_xh_zl = drifters_xh_zl2.assign(lon_gridded = (np.ceil(drifters.lon) - 0.5))
# drifters_xh_zl

### <font color="F907035"> add `xh` and `zl` coordinates s.t. `xh(i)` and `zl(i)`. </font>
    
### <font color="F907035"> now there are 3 coordinates: (valid_time, xh, zl). </font>

### <font color="F907035"> create multiindex using the 3 coordinates to add `xh`, `zl`, `time` dimensions. This allows for datavariables to depend on (xh,zl,time) instead of (i). </font>

In [35]:
drifters_xh_zl_coords = drifters_xh_zl.set_coords(['xh','zl_appx'])
xhs = drifters_xh_zl_coords.xh.values
zls = drifters_xh_zl_coords.zl_appx.values
times = drifters_xh_zl_coords.valid_time.values
ds1 = drifters_xh_zl_coords.drop(['xh','zl_appx','valid_time'])
# make new coordinate, i, using multiindex (combining xh & zl arrays)
ds1.coords['i'] = pd.MultiIndex.from_arrays([xhs,zls,times],names=['xh','zl_appx','time'])

# unstack ds to remove coordinate i but retain indices xh & zl
drifters_coords2 = ds1.unstack()
# len(np.unique(drifters_coords.drifter_num))
# drifters_coords2

/tmp/ipykernel_2363007/1244380540.py:7: FutureWarning: the `pandas.MultiIndex` object(s) passed as 'i' coordinate(s) or data variable(s) will no longer be implicitly promoted and wrapped into multiple indexed coordinates in the future (i.e., one coordinate for each multi-index level + one dimension coordinate). If you want to keep this behavior, you need to first wrap it explicitly using `mindex_coords = xarray.Coordinates.from_pandas_multiindex(mindex_obj, 'dim')` and pass it as coordinates, e.g., `xarray.Dataset(coords=mindex_coords)`, `dataset.assign_coords(mindex_coords)` or `dataarray.assign_coords(mindex_coords)`.
  ds1.coords['i'] = pd.MultiIndex.from_arrays([xhs,zls,times],names=['xh','zl_appx','time'])


In [36]:
zl_correct = [6.5,   20.5,   36.5,   55. ,   76.5,  101. ,  129. ,  161. ,  197.5,
        239.5,  287.5,  342. ,  403.5,  473. ,  551.5,  639.5,  738.5,  849.5,
        973. , 1110. , 1261. , 1426.5, 1607. , 1803. , 2014.5, 2240.5, 2480.5,
       2733.5, 2997.5, 3270.5, 3550. , 3845.]
drifters_coords2.coords['zl'] = drifters_coords2.zl_appx - drifters_coords2.zl_appx + zl_correct
drifters_coords = drifters_coords2.swap_dims({'zl_appx':'zl'}).drop('zl_appx')
drifters_coords = drifters_coords[['xh','zl','time','lon','lat','k','depth','year','day','drifter_num',
                                   'id_cnt','id_ij','theta','uvel','vvel','depth_gridded','lon_gridded']]
drifters_coords

<xarray.Dataset>
Dimensions:        (xh: 80, zl: 32, time: 4381)
Coordinates:
  * xh             (xh) float64 0.5 1.5 2.5 3.5 4.5 ... 75.5 76.5 77.5 78.5 79.5
  * zl             (zl) float64 6.5 20.5 36.5 ... 3.27e+03 3.55e+03 3.845e+03
  * time           (time) float64 0.06944 5.0 13.0 ... 2.206e+04 2.206e+04
Data variables: (12/14)
    lon            (xh, zl, time) float64 0.5 0.5 0.5 0.5 ... nan nan nan nan
    lat            (xh, zl, time) float64 -10.5 -10.5 -10.5 ... nan nan nan
    k              (xh, zl, time) float64 0.0 0.0 0.0 0.0 ... nan nan nan nan
    depth          (xh, zl, time) float64 0.0 0.0 0.0 0.0 ... nan nan nan nan
    year           (xh, zl, time) float64 1.499e+03 1.499e+03 ... nan nan
    day            (xh, zl, time) float64 356.1 361.0 1.0 6.0 ... nan nan nan
    ...             ...
    id_ij          (xh, zl, time) float64 4.721e+03 4.721e+03 ... nan nan
    theta          (xh, zl, time) float64 15.92 15.92 15.92 ... nan nan nan
    uvel           (xh, zl, time) float64 0.0 0.0 0.0 0.0 ... nan nan nan nan
    vvel           (xh, zl, time) float64 0.0 0.0 0.0 0.0 ... nan nan nan nan
    depth_gridded  (xh, zl, time) float64 6.5 6.5 6.5 6.5 ... nan nan nan nan
    lon_gridded    (xh, zl, time) float64 0.5 0.5 0.5 0.5 ... nan nan nan nan
Attributes:
    file_format_major_version:  0
    file_format_minor_version:  1

In [37]:
drifters_coords_t0 = drifters_coords.isel(time=slice(0,1))
drifters_coords_t1 = drifters_coords.isel(time=slice(1,2))

# <mark style="background-color: #E7C5FF"> <font color ='7C1931'> Merge drifters_coords dataset with areas dataset to get volume </font></mark>
    
# <mark style="background-color: #E7C5FF"> <font color ='7C1931'>  transport assigned to each individual drifter. Datasets will be merged by </font></mark>
    
# <mark style="background-color: #E7C5FF"> <font color ='7C1931'>   aliging along the same (xh, zl, time) dimensions. </font> </mark>


## <mark style="background-color: #E7C5FF"> <font color ='7C1931'> Combine area \& drifters data into one dataset based on (xh,zl,time). </font></mark>

### <font color='7C1931'> Individual particle volume transport will be constant for all time. Calculate area from `ocean_geo` data. Convert to dataset. Since `area_ds_unstacked` has coordinates \& dimensions (xh,zl,time), we can merge this dataset with `drifters_coords` (dimensions and coordinates match).</font>

In [38]:
# eliminate the need to include t=0 for vvel from drifters_coords_t0.
# so try making area_ds based on drifters_coords_t1
# calculate area based on grid cell size at 10S

area_on_xh_zl1 = ocean_geo.dxCv.sel(yq=-10,method='nearest') * prog.h.isel(Time=-1).sel(yh=-10.5,method='nearest')
area_on_xh_zl = area_on_xh_zl1.drop('Time')

# convert dataarray to dataset
area_ds = area_on_xh_zl.to_dataset(name='area')
area_ds = area_ds.assign_coords(time = drifters_coords_t1.time)

xhs = area_ds.xh.values
zls = area_ds.zl.values
times = area_ds.time.values

area_ds_stacked = area_ds.stack(xhzltime = ('xh','zl','time'))
area_ds_stacked = area_ds.stack(xhzl = ('xh','zl'))
len(np.unique(area_ds_stacked.area))
area_ds_unstacked = area_ds_stacked.unstack().drop('yq').drop('yh')
area_ds_unstacked

drifters_and_areas = xr.merge([drifters_coords,area_ds_unstacked],join='inner')
# remove any drifters not present at t=0
# this way, there aren't any unassigned drifters since # of drifters at t=0 =/= # of drifters at t= some other time
dns_at_t0 = np.unique(drifters_and_areas.drifter_num.isel(time=0))[:-1]
drifters_and_areas = drifters_and_areas.where(drifters_and_areas.drifter_num.isin(dns_at_t0))
#len(np.unique(drifters_and_areas.drifter_num))

drifters_and_areas1 = xr.merge([drifters_coords_t1,drifters_and_areas])

## <mark style="background-color: #E7C5FF"> <font color ='7C1931'> Begin the conversion from (xh,zl,time) coordinates to (i,time) coordinates. </font></mark>

### <font color ='7C1931'> Since we have one drifter/grid cell, each drifter, at t=0, has a unique (xh,zl) coordinate. Each drifter at t=0 also has a unique drifter number. Therefore, we can use a `swap_dims` to convert from (xh,zl) dimensions to i=drifter_num dimensions. </font>

In [39]:
stacked = drifters_and_areas1.stack(xhzl = ('xh','zl'))
dn_for_coords = stacked.drifter_num.isel(time=0).drop('time')
stacked_plus_dn = stacked.assign_coords(i = dn_for_coords)
stacked_plus_dn
stacked_i_coord = stacked_plus_dn.swap_dims({'xhzl': 'i'})

### <font color ='7C1931'> Get drifter meridional velocities at t=5 to calculate volume transport. </font>

In [40]:
getvvelt5 = drifters_and_areas1.stack(xhzl = ('xh','zl'))

getvvelt5_plus_dn = getvvelt5.assign_coords(i=dn_for_coords)
getvvelt5_i_coord = getvvelt5_plus_dn.swap_dims({'xhzl':'i'})

## 5-23 use getvvelt5_i_coord.vvel to calculate Sv
stacked_i_coord = stacked_i_coord.assign(Sv = (stacked_i_coord.area * getvvelt5_i_coord.isel(time=0).vvel) / (10**6))

### <font color ='7C1931'> Reattach the rest of the times (recall we only used t=0 and t=5). This requires converting the rest of the time data into (i=drifter_num, time) coordinates.</font>

In [41]:
drifters_xh_zl_coords = drifters_xh_zl.set_coords(['valid_time','drifter_num'])
drifters_xh_zl_coords = drifters_xh_zl_coords.assign(drifter_num1 = drifters_xh_zl_coords.drifter_num)

dns = drifters_xh_zl_coords.drifter_num.values
times = drifters_xh_zl_coords.valid_time.values

therest_ds = drifters_xh_zl_coords.drop(['drifter_num','valid_time'])

# new coord with MultiIndex
therest_ds['i'] = pd.MultiIndex.from_arrays([times,dns],names=['time','dn'])

therest_ds_coords = therest_ds.unstack()
therest_ds_coords_i = therest_ds_coords.rename({'dn':'i'})
puhlease = therest_ds_coords_i.where(therest_ds_coords_i.time>1).dropna('time',how='all')
puhlease = puhlease.assign(drifter_num = puhlease.drifter_num1).drop(['drifter_num1'])

### <font color ='7C1931'> Drop `MultiIndex` dimensions to properly align the t=0 drifters with Sv and the rest of the data. Then expand dimensions so for each drifter, at every timestep, $Sv_{t=n} = Sv_{t=0}$. </font>

In [42]:
drop1 = stacked_i_coord.drop('xhzl')
drop2 = drop1.drop('xh')
drop3 = drop2.drop('zl')
drop3.dropna('i')

maybe1 = xr.merge([drop3.dropna('i'),puhlease])
len(np.unique(maybe1.drifter_num))
maybe1

# maybe1['Sv'] = maybe1.Sv.expand_dims(dim={'time':maybe1.time})
# maybe1['init_depths'] = maybe1.depth.isel(time=0).expand_dims({'time':maybe1.time})
# maybe1['init_lons'] = maybe1.lon.isel(time=0).expand_dims({'time':maybe1.time})
# maybe1

maybe1 = maybe1.assign(Sv_use = maybe1.Sv.isel(time=0))
maybe1['Sv_use']=maybe1.Sv_use.expand_dims(dim={'time':maybe1.time})
maybe1 = maybe1.drop('Sv')
maybe1['Sv'] = maybe1.Sv_use
maybe1 = maybe1.drop('Sv_use')
maybe1['init_depths'] = maybe1.depth.isel(time=0).expand_dims({'time':maybe1.time})
maybe1['init_lons'] = maybe1.lon.isel(time=0).expand_dims({'time':maybe1.time})
maybe1

<xarray.Dataset>
Dimensions:        (time: 4380, i: 1280)
Coordinates:
  * time           (time) float64 5.0 13.0 18.0 ... 2.206e+04 2.206e+04
  * i              (i) float64 1.0 2.0 3.0 4.0 ... 1.278e+03 1.279e+03 1.28e+03
Data variables: (12/20)
    lon            (time, i) float64 0.5 0.5 0.5 0.5 0.5 ... 39.5 39.5 39.5 39.5
    lat            (time, i) float64 -10.5 -10.5 -10.5 ... -10.5 -10.5 -10.5
    k              (time, i) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    depth          (time, i) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    year           (time, i) float64 1.499e+03 1.499e+03 ... 1.559e+03 1.559e+03
    day            (time, i) float64 361.0 361.0 361.0 ... 341.0 341.0 341.0
    ...             ...
    area           (time, i) float64 nan nan nan nan nan ... nan nan nan nan nan
    xh             (time, i) float64 0.5 0.5 0.5 0.5 0.5 ... 39.5 39.5 39.5 39.5
    zl_appx        (time, i) float64 6.5 6.5 6.5 6.5 6.5 ... 6.5 6.5 6.5 6.5 6.5
    Sv             (time, i) float64 nan nan nan nan nan ... nan nan nan nan 0.0
    init_depths    (time, i) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    init_lons      (time, i) float64 0.5 0.5 0.5 0.5 0.5 ... 39.5 39.5 39.5 39.5
Attributes:
    file_format_major_version:  0
    file_format_minor_version:  1

# <mark style="background-color: #EEF47D"> <font color ='F97O35'> Get the full trajectories of only particles that _start_ above 1000m </font> </mark>

In [43]:
# get only the full trajectories of only particles that START above 1000m 
maybe1_t0 = maybe1.isel(time=0)
maybe1_above_1000mt0 =maybe1_t0.where(maybe1_t0.depth<=1000).dropna('i',how='all')
maybe1_full_traj = maybe1.where(maybe1_above_1000mt0.drifter_num.isin(maybe1.i.values)).dropna('i',how='all')

# get only first 2190 times to match KD = 5E-6
# maybe1_full_traj = maybe1_full_traj0.isel(time=slice(0,2190))

In [44]:
# maybe1_full_traj.to_netcdf('./7-17_reach20N_above1000m_mintime_cross_lats/7-23_TD_KD1E-5_above1000m_KD1E-5.nc',format='NETCDF4',mode='w')

# <mark style="background-color: #EEF47D"> <font color ='F97O35'> Of the particles that start above 1000m, get the particles that cross 20.5$^{\circ}$N </font> </mark>

In [45]:
# of the particles that start above 1000m, get only the particles that cross 20.5N
chosen_lat = 20.5
moving_n_at_t5 = maybe1.where((maybe1.vvel > 0) & (maybe1.time ==5)).dropna('i',how='all')
ivals = moving_n_at_t5.dropna('time',how='all').i.values
moving_n_sincet5 = maybe1.where(maybe1.i.isin(ivals)).dropna('i',how='all')
moving_n = moving_n_sincet5.where(moving_n_sincet5.vvel>0)
moving_n_chosen_lat = moving_n.where(moving_n.lat >= chosen_lat).dropna('i',how='all')
predel_min_time_ind = moving_n_chosen_lat.drifter_num.argmin(dim='time').dropna('i').to_numpy()
predel_i = moving_n_chosen_lat.drifter_num.argmin(dim='time').i.values
predel_i_nans_ind = np.argwhere(np.isnan(predel_i))
dn_times_del = np.delete(predel_min_time_ind,predel_i_nans_ind).astype(int)
is_inds = np.argwhere(~np.isnan(moving_n_chosen_lat.i.values))


mintimeind = moving_n_chosen_lat.lat.argmin(dim='time').values

lon_list,lat_list,depth_list,drifter_num_list,vvel_list,depth_gridded_list,lon_gridded_list,Sv_list,theta_list =[],[],[],[],[],[],[],[],[]
for i in range(0,len(mintimeind)):
    lon_list.append(moving_n_chosen_lat['lon'][mintimeind[i],i])
    lat_list.append(moving_n_chosen_lat['lat'][mintimeind[i],i])
    depth_list.append(moving_n_chosen_lat['depth'][mintimeind[i],i])
    drifter_num_list.append(moving_n_chosen_lat['drifter_num'][mintimeind[i],i])
    vvel_list.append(moving_n_chosen_lat['vvel'][mintimeind[i],i])
    depth_gridded_list.append(moving_n_chosen_lat['depth_gridded'][mintimeind[i],i])
    lon_gridded_list.append(moving_n_chosen_lat['lon_gridded'][mintimeind[i],i])
    Sv_list.append(moving_n_chosen_lat['Sv'][mintimeind[i],i])
    theta_list.append(moving_n_chosen_lat['theta'][mintimeind[i],i])
    
dataset_names = [1,2,3,4,5,6,7,8,9]
list_names = [lon_list,lat_list,depth_list,drifter_num_list,vvel_list,depth_gridded_list,lon_gridded_list,Sv_list,theta_list]
datavar_names = ['lon','lat','depth','drifter_num','vvel','depth_gridded','lon_gridded','Sv','theta']
for i in range(0,9):
    dataset_names[i] = xr.concat(list_names[i],dim='i').to_dataset(name=datavar_names[i])
    
mintime_ds = xr.merge([dataset_names[0],dataset_names[1],dataset_names[2],dataset_names[3],
          dataset_names[4],dataset_names[5],dataset_names[6],dataset_names[7],dataset_names[8]])
mintime_ds

# we have the i values of the particles that make it to 20.5N. Use it to subset all the particles that 
# are above 1000m
full_traj_20N = maybe1.where(maybe1_above_1000mt0.drifter_num.isin(mintime_ds.i.values)).dropna('i',how='all')
maybe3 = full_traj_20N

## <font color ='F97O35'> For each specified latitude, find the depth \& latitude (and corresponding datavariables) of each particle as it crosses over the latitude for the first time. </font>

In [46]:
## get individual datasets for each latitude
chosen_lats_array = [-10.5,-0.5,5.5,10.5,20.5]
mintime_ds_names = ['a','b','c','d','e']
for k in range(0,5):
    moving_n_at_t5 = maybe3.where((maybe3.vvel > 0) & (maybe3.time ==5)).dropna('i',how='all')
    ivals = moving_n_at_t5.dropna('time',how='all').i.values
    moving_n_sincet5 = maybe3.where(maybe3.i.isin(ivals)).dropna('i',how='all')
    moving_n = moving_n_sincet5.where(moving_n_sincet5.vvel>0)
    moving_n_chosen_lat = moving_n.where(moving_n.lat >= chosen_lats_array[k]).dropna('i',how='all')
    predel_min_time_ind = moving_n_chosen_lat.drifter_num.argmin(dim='time').dropna('i').to_numpy()
    predel_i = moving_n_chosen_lat.drifter_num.argmin(dim='time').i.values
    predel_i_nans_ind = np.argwhere(np.isnan(predel_i))
    dn_times_del = np.delete(predel_min_time_ind,predel_i_nans_ind).astype(int)
    is_inds = np.argwhere(~np.isnan(moving_n_chosen_lat.i.values))
    mintimeind = moving_n_chosen_lat.lat.argmin(dim='time').values

    lon_list,lat_list,depth_list,drifter_num_list,vvel_list,depth_gridded_list,lon_gridded_list,Sv_list,theta_list =[],[],[],[],[],[],[],[],[]
    for i in range(0,len(mintimeind)):
        lon_list.append(moving_n_chosen_lat['lon'][mintimeind[i],i])
        lat_list.append(moving_n_chosen_lat['lat'][mintimeind[i],i])
        depth_list.append(moving_n_chosen_lat['depth'][mintimeind[i],i])
        drifter_num_list.append(moving_n_chosen_lat['drifter_num'][mintimeind[i],i])
        vvel_list.append(moving_n_chosen_lat['vvel'][mintimeind[i],i])
        depth_gridded_list.append(moving_n_chosen_lat['depth_gridded'][mintimeind[i],i])
        lon_gridded_list.append(moving_n_chosen_lat['lon_gridded'][mintimeind[i],i])
        Sv_list.append(moving_n_chosen_lat['Sv'][mintimeind[i],i])
        theta_list.append(moving_n_chosen_lat['theta'][mintimeind[i],i])
    dataset_names = [1,2,3,4,5,6,7,8,9]
    list_names = [lon_list,lat_list,depth_list,drifter_num_list,vvel_list,depth_gridded_list,lon_gridded_list,Sv_list,theta_list]
    datavar_names = ['lon','lat','depth','drifter_num','vvel','depth_gridded','lon_gridded','Sv','theta']
    for i in range(0,9):
        dataset_names[i] = xr.concat(list_names[i],dim='i').to_dataset(name=datavar_names[i])

    mintime_ds_names[k] = xr.merge([dataset_names[0],dataset_names[1],dataset_names[2],dataset_names[3],
              dataset_names[4],dataset_names[5],dataset_names[6],dataset_names[7],dataset_names[8]])

In [47]:
mintime_ds_10S = mintime_ds_names[0]
mintime_ds_0N = mintime_ds_names[1]
mintime_ds_5N = mintime_ds_names[2]
mintime_ds_10N = mintime_ds_names[3]
mintime_ds_20N = mintime_ds_names[4]
mintime_ds_array = [mintime_ds_10S,mintime_ds_0N,mintime_ds_5N,mintime_ds_10N,mintime_ds_20N]

In [48]:
mintime_ds_array[0].lat.max()

<xarray.DataArray 'lat' ()>
array(-5.88009203)

In [49]:
# # 6-5 convert to netcdf to open in other notebook for KD5E-6

# mintime_ds_10S.to_netcdf('./7-17_reach20N_above1000m_mintime_cross_lats/7-23_TD_KD1E-5_reach20N_above1000m_mintime_ds_10S_KD1E-5.nc',format='NETCDF4',mode='w')
# mintime_ds_0N.to_netcdf('./7-17_reach20N_above1000m_mintime_cross_lats/7-23_TD_KD1E-5_reach20N_above1000m_mintime_ds_0N_KD1E-5.nc',format='NETCDF4',mode='w')
# mintime_ds_5N.to_netcdf('./7-17_reach20N_above1000m_mintime_cross_lats/7-23_TD_KD1E-5_reach20N_above1000m_mintime_ds_5N_KD1E-5.nc',format='NETCDF4',mode='w')
# mintime_ds_10N.to_netcdf('./7-17_reach20N_above1000m_mintime_cross_lats/7-23_TD_KD1E-5_reach20N_above1000m_mintime_ds_10N_KD1E-5.nc',format='NETCDF4',mode='w')
# mintime_ds_20N.to_netcdf('./7-17_reach20N_above1000m_mintime_cross_lats/7-23_TD_KD1E-5_reach20N_above1000m_mintime_ds_20N_KD1E-5.nc',format='NETCDF4',mode='w')
